## Retirement Calculator
Planning for retirement is incredibly important. Try changing the parameters to 

In [11]:
import numpy as np

# DEFAULT PARAMETERS

# starting age or current year
STARTING_YEAR = 21
STARTING_PRINCIPAL = 25000
INITIAL_ANNUAL_INVESTMENT = 20000
ANNUAL_INVESTMENT_INCREASE_LAMBDA = lambda previous: previous * 1.02 + 2000

INITIAL_FRACTION_STOCK = 0.95
INITIAL_FRACTION_BONDS = 0.05
INITIAL_FRACTION_OTHER = 0.00
FRACTION_CHANGE_LAMBDA = lambda stk, bon, oth: (stk - 0.005, bon + 0.005, oth)

STOCK_GROWTH = 1.075
BONDS_GROWTH = 1.03
OTHER_GROWTH = 1.0

TAX_RATE = 0.35
INFLATION = 1.022

# target in current dollars, inflation adjusted, post tax
TARGET = 2e6

In [13]:
def fmt(string, spacing, year=False, text=False):
    if text:
        if year:
            return f'{"{: <" + str(spacing) + "}"}'.format(string)
        return f'{"{: >" + str(spacing) + "}"}'.format(string)
        
    if year:
        return f'{"{: <" + str(spacing) + "}"}'.format('{:.0f}'.format(string))
    return f'{"{: >" + str(spacing) + "}"}'.format('{:,.0f}'.format(string))

def compute_yearly_growth(after_inflation=False):
    stock_frac = INITIAL_FRACTION_STOCK
    bonds_frac = INITIAL_FRACTION_BONDS
    other_frac = INITIAL_FRACTION_OTHER
    
    stock = STARTING_PRINCIPAL * stock_frac
    bonds = STARTING_PRINCIPAL * bonds_frac
    other = STARTING_PRINCIPAL * other_frac
    
    investment = INITIAL_ANNUAL_INVESTMENT
    year = STARTING_YEAR
    
    output = fmt('Year', 4, year=True, text=True)
    output = output + fmt('Principal (eff)', 17, text=True)
    output = output + fmt('Principal (true)', 17, text=True)
    output = output + fmt('Investment', 15, text=True)
    output = output + fmt('Stock', 17, text=True)
    output = output + fmt('Bonds', 17, text=True)
    output = output + fmt('Other', 14, text=True)
    output = output + fmt('Inflation', 12, text=True)
    print(output)
    
    running = True
    while running:
        assert (stock_frac + bonds_frac + other_frac - 1.0) < 1e-6
        assert stock_frac >= 0.0 and bonds_frac >= 0.0 and other_frac >= 0.0

        total = stock + bonds + other
        inflation = np.power(INFLATION, (year - STARTING_YEAR))
        effective = total/inflation * (1 - TAX_RATE)
        
        output = fmt(year, 4, year=True)
        output = output + fmt(effective, 17)
        output = output + fmt(total, 17)
        
        if after_inflation:
            output = output + fmt(investment/inflation, 15)
            output = output + fmt(stock/inflation, 17)
            output = output + fmt(bonds/inflation, 17)
            output = output + fmt(other/inflation, 14)
        else:
            output = output + fmt(investment, 15)
            output = output + fmt(stock, 17)
            output = output + fmt(bonds, 17)
            output = output + fmt(other, 14)
            
        output = output + fmt(f'{np.round(inflation, 2)}x', 12, text=True)
        
        print(output)
        running = (effective < TARGET)
        
        stock_frac, bonds_frac, other_frac = FRACTION_CHANGE_LAMBDA(stock_frac, bonds_frac, other_frac)
        
        total = stock * STOCK_GROWTH + bonds * BONDS_GROWTH + other * OTHER_GROWTH + investment
        stock = total * stock_frac
        bonds = total * bonds_frac
        other = total * other_frac
        
        investment = ANNUAL_INVESTMENT_INCREASE_LAMBDA(investment)
        year += 1

In [23]:
# NON-DEFAULT PARAMETERS

STARTING_PRINCIPAL = 0
INITIAL_ANNUAL_INVESTMENT = 15000
ANNUAL_INVESTMENT_INCREASE_LAMBDA = lambda previous: previous * 1.02 + 500

INITIAL_FRACTION_STOCK = 0.85
INITIAL_FRACTION_BONDS = 0.10
INITIAL_FRACTION_OTHER = 0.05
FRACTION_CHANGE_LAMBDA = lambda stk, bon, oth: (stk, bon, oth)

STOCK_GROWTH = 1.07
BONDS_GROWTH = 1.03
OTHER_GROWTH = 1.05

TAX_RATE = 0.30
INFLATION = 1.02

TARGET = 1.5e6

compute_yearly_growth()

Year  Principal (eff) Principal (true)     Investment            Stock            Bonds         Other   Inflation
21                  0                0         15,000                0                0             0        1.0x
22             10,294           15,000         15,800           12,750            1,500           750       1.02x
23             21,379           31,775         16,616           27,009            3,178         1,589       1.04x
24             33,282           50,456         17,448           42,888            5,046         2,523       1.06x
25             46,034           71,184         18,297           60,507            7,118         3,559       1.08x
26             59,666           94,109         19,163           79,992            9,411         4,705        1.1x
27             74,210          119,389         20,046          101,481           11,939         5,969       1.13x
28             89,700          147,196         20,947          125,116           14,720 